## Titanic Visualization using D3.js

By Grace Cho

The visualizations explore the relationship between survival rates of passengers aboard Titanic and passenger information such as age. From the analysis, you can see the following: 1) lower class (i.e. 3) has lower survival rate, 2) while for females, older age groups have higher survival rates, for males, survival rates range around 20% for adults. The dataset contains demographics and passenger information from a subset of 2224 passengers and crew onboard the Titanic. More information about dataset can be found [here](www.kaggle.com/c/titanic).

### Visualizations

![](http://i.imgur.com/OXGYovC.png)
![](http://i.imgur.com/MOIne2f.png)
![](http://i.imgur.com/3ZyWiqg.png)

### Design

Following details on iterations explain the design choices I made before and after collecting feedback.

In the 1st iteration, I drew 2 bar charts showing survival rates by passenger class and age group. As both class and age group were categorical variables, bar charts made most sense (note: age, a continuous variable was grouped for analysis). Next, to explore whether there is a difference in survival rate between females and males, I added the sex variable to the chart. Bars were stacked next to one another for easy comparison.

In the 2nd iteration, interactivity was added. Instead of simply displaying the legends, now if viewers wanted to focus only on females for example, they can click and change the chart accordingly.

In the final iteration, a 3rd chart was added to take a closer look at survival rate by age groups. Here I chose the bubble graph to easily show how the age groups compare to one another. Unlike the 2nd chart, the chart animates back and forth automatically to emphasize the wide disparity between females and males. Viewers have the option to click bars on the right and pause the animation.

### index.html

In [ ]:
<!DOCTYPE html>
<html lang = "en">
    <head>
        <meta charset="utf-8">
        <script src="http://d3js.org/d3.v3.min.js"></script>
        <script src="http://dimplejs.org/dist/dimple.v2.2.0.min.js"></script>

        <style>
        .chart, h1, h5 {
            text-align: center;
        }

        div.buttons {
          float: right;
          margin-top: 25px;
        }

        div.buttons div {
          background-color: lightgrey;
          color: white;
          padding: 7px;
        }
        </style>
    </head>
    <body>
        <h1>Titanic Passengers: who survived?</h1>
        <h5>By: Grace Cho</h4>
        <hr width="60%" align="center">
        <br>
        <!-- 1st chart -->
        <div class = "chart" id="chartContainer">
        <script type="text/javascript">
            var width = 600,
                height = 450,
                margin = 50;

            var svg = d3.select("#chartContainer")
                        .append("svg")
                        .attr("width", width + margin)
                        .attr("height", height + margin);

            // Create survival rate by passenger class chart
            d3.csv("/data/sex_class_survival_rate.csv", function(data) {
                var chart = new dimple.chart(svg, data);
                var x = chart.addCategoryAxis("x", ["Pclass", "Sex"]);
                x.addGroupOrderRule(["female", "male"]);
                x.title = "Passenger Class";
                var y = chart.addMeasureAxis("y", "Survival_rate");
                y.title = "Survival Rate"
                chart.addSeries("Sex", dimple.plot.bar);
                chart.setBounds(100,90,420,370);
                var myLegend = chart.addLegend(550, 100, 60, 300, "Right");

                // Add chart title
                svg.append("text")
                   .attr("x", (width + margin) / 2)
                   .attr("y", 50)
                   .style("text-anchor", "middle")
                   .style("font-weight", "bold")
                   .style("font-size", "22px")
                   .text("Survival Rate by Passenger Class");

               chart.draw();

               // Orphan the legend
               chart.legends = [];

               // Add legend guide
               svg.selectAll("title_text")
                     .data(["Click legend to","show/hide gender:"])
                     .enter()
                     .append("text")
                       .attr("x", 540)
                       .attr("y", function (d, i) { return 90 + i * 14; })
                       .style("font-family", "sans-serif")
                       .style("font-size", "10px")
                       .style("color", "Black")
                       .text(function (d) { return d; });

               // Get a unique list of gender values to use when filtering
               var filterValues = dimple.getUniqueValues(data, "Sex");
               // Get all the rectangles from orphaned legend
               myLegend.shapes.selectAll("rect")
                 // Add a click event to each rectangle
                 .on("click", function (e) {
                   // Below indicates whether the item is already visible or not
                   var hide = false;
                   var newFilters = [];
                   // If filters contain the clicked shape, hide it
                   filterValues.forEach(function (f) {
                     if (f === e.aggField.slice(-1)[0]) {
                       hide = true;
                     } else {
                       newFilters.push(f);
                     }
                   });
                   // Hide the shape or show it
                   if (hide) {
                     d3.select(this).style("opacity", 0.2);
                   } else {
                     newFilters.push(e.aggField.slice(-1)[0]);
                     d3.select(this).style("opacity", 0.8);
                   }
                   // Update the filters
                   filterValues = newFilters;
                   // Filter the data
                   chart.data = dimple.filterData(data, "Sex", filterValues);
                   // Passing a duration parameter makes the chart animate
                   chart.draw(800);
                 });
             });

        </script>
        </div>
        <br>
        <!-- 2nd chart -->
        <div class = "chart" id="chartContainer1">
            <script type="text/javascript">
                var width = 600,
                    height = 450,
                    margin = 50;

                var svg1 = d3.select("#chartContainer1")
                            .append("svg")
                            .attr("width", width + margin)
                            .attr("height", height + margin);

                // Create survival rate by age groups chart
                d3.csv("/data/sex_age_survival_rate.csv", function(data) {
                    var chart = new dimple.chart(svg1, data);
                    chart.setBounds(100,90,420,370);

                    var x = chart.addCategoryAxis("x", ["age_group", "sex"]);
                    x.addOrderRule("order");
                    x.title = "Age Groups";
                    var y = chart.addMeasureAxis("y", "survival_rate");
                    y.title = "Survival Rate";
                    chart.addSeries("sex", dimple.plot.bar);
                    var myLegend = chart.addLegend(550, 100, 60, 300, "Right");

                    // Add title chart
                    svg1.append("text")
                       .attr("x", (width + margin) / 2)
                       .attr("y", 50)
                       .style("text-anchor", "middle")
                       .style("font-weight", "bold")
                       .style("font-size", "22px")
                       .text("Survival Rate by Age: An Overview");

                    chart.draw();

                    // Orphan the legend
                    chart.legends = [];

                    // Add legend guide
                    svg1.selectAll("title_text")
                          .data(["Click legend to","show/hide gender:"])
                          .enter()
                          .append("text")
                            .attr("x", 540)
                            .attr("y", function (d, i) { return 90 + i * 14; })
                            .style("font-family", "sans-serif")
                            .style("font-size", "10px")
                            .style("color", "Black")
                            .text(function (d) { return d; });

                    // Get a unique list of gender values to use when filtering
                    var filterValues = dimple.getUniqueValues(data, "sex");
                    // Get all the rectangles from orphaned legend
                    myLegend.shapes.selectAll("rect")
                      // Add a click event to each rectangle
                      .on("click", function (e) {
                        // Indicates whether the item is already visible or not
                        var hide = false;
                        var newFilters = [];
                        // If the filters contain the clicked shape, hide it
                        filterValues.forEach(function (f) {
                          if (f === e.aggField.slice(-1)[0]) {
                            hide = true;
                          } else {
                            newFilters.push(f);
                          }
                        });
                        // Hide the shape or show it
                        if (hide) {
                          d3.select(this).style("opacity", 0.2);
                        } else {
                          newFilters.push(e.aggField.slice(-1)[0]);
                          d3.select(this).style("opacity", 0.8);
                        }
                        // Update the filters
                        filterValues = newFilters;
                        // Filter the data
                        chart.data = dimple.filterData(data, "sex", filterValues);
                        // Passing a duration parameter makes the chart animate
                        chart.draw(800);
                      });
                  });
        </script>
        </div>
        <br>
        <br>
        <!-- 3rd chart -->
        <div class = "chart" id="chartContainer2">
            <script type = "text/javascript">

            var width = 650,
                height = 450,
                margin = 50;

            var svg2 = d3.select("#chartContainer2")
                        .append("svg")
                        .attr("width", width + margin)
                        .attr("height", height + margin);

            // Create survival rate by age groups chart (v2: closer look)
            d3.csv("/data/sex_age_survival_rate.csv", function(data) {

                // Create indicator chart on right of main chart
                var indicator = new dimple.chart(svg2, data);

                // Pick deafult and selected colors
                var defaultColor = indicator.defaultColors[0];
                var indicatorColor = indicator.defaultColors[2];

                // Frame duration for the animation in milliseconds
                var frame = 2000;
                var firstTick = true;

                // Place indicator bar chart to the right
                indicator.setBounds(540, 75, 153, 311);

                // Add variable along y-axis of indicator
                var y = indicator.addCategoryAxis("y", "sex");
                // Add variable for x-axis & hide axis
                var x = indicator.addMeasureAxis("x", "sum_survived");
                x.hidden = true;

                // Add bars to indicator & add event handlers
                var s = indicator.addSeries(null, dimple.plot.bar);
                s.addEventHandler("click", onClick);
                indicator.draw();

                // Remove title and lines from y-axis
                y.titleShape.remove();
                y.shapes.selectAll("line, path").remove();

                // Move text into rectangles
                y.shapes.selectAll("text")
                        .style("text-anchor", "start")
                        .style("font-size", "11px")
                        .attr("transform", "translate(18, 0.5)");

                // Add legend guide
                svg2.selectAll("title_text")
                      .data(["Click bar to select",
                          "and pause. Click again",
                          "to resume animation"])
                      .enter()
                      .append("text")
                      .attr("x", 540)
                      .attr("y", function (d, i) { return 58 + i * 12; })
                      .style("font-family", "sans-serif")
                      .style("font-size", "10px")
                      .style("color", "Black")
                      .text(function (d) { return d; });

                // Style the rectangles
                s.shapes
                        .attr("rx",10)
                        .attr("ry", 10)
                        .style("opacity", 0.4);

                // Create survival rate by sum of survivors chart
                var bubbles = new dimple.chart(svg2, data);
                bubbles.setBounds(100,90,420,370);
                var x = bubbles.addMeasureAxis("x", "sum_passengers");
                x.title = "Number of Passengers";
                var y = bubbles.addMeasureAxis("y", "survival_rate");
                y.title = "Survival Rate";
                var mySeries = bubbles.addSeries("age_group", dimple.plot.bubble);

                // Create and change legend order
                var myLegend = bubbles.addLegend(100,50,410,60);
                // Access legend
                myLegend._getEntries = function() {
                    var orderedValues = ["1-10", "11-20", "21-30", "31-40", "41-50", "51-60", "61-70", "71+", "No Data"];
                    var entries = [];
                    orderedValues.forEach(function(v) {
                        entries.push({
                            key : v,
                            fill : bubbles.getColor(v).fill,
                            stroke : bubbles.getColor(v).stroke,
                            opacity : bubbles.getColor(v).opacity,
                            series : [mySeries],
                            aggField : [v]
                        });
                    }, this);
                    return entries;
                };

                // Add chart title
                svg2.append("text")
                   .attr("x", (width + margin) / 2)
                   .attr("y", 20)
                   .style("text-anchor", "middle")
                   .style("font-weight", "bold")
                   .style("font-size", "22px")
                   .text("Survival Rate by Age: A Closer Look");

                // Add a storyboard to the main chart and set the tick event
                var story = bubbles.setStoryboard("sex", onTick);
                // Change the frame duration
                story.frameDuration = frame;
                // Draw bubble chart
                bubbles.draw();

                // Orphan legends
                bubbles.legend = [];
                // Remove storylabels
                story.storyLabel.remove();

                // Create storyboard control functions
                function onClick(e) {
                    story.pauseAnimation();
                    if (e.yValue === story.getFrameValue()) {
                        story.startAnimation();
                    }   else {
                        story.goToFrame(e.yValue);
                        story.pauseAnimation();
                    }
                }

                function onTick(e) {
                    if (!firstTick) {
                        s.shapes
                            .transition()
                            .duration(frame/2)
                            .style("fill", function (d) { return (d.y === e ? indicatorColor.fill : defaultColor.fill) })
                            .style("stroke", function (d) { return (d.y === e ? indicatorColor.stroke : defaultColor.stroke) });
                    }
                    firstTick = false;
                }

            });
            </script>
         </div>
    </body>
</html>

---
#### Resources
https://discussions.udacity.com/t/dynamic-buttons-between-cabin-class-and-sex/177482/14
http://dimplejs.org/advanced_examples_viewer.html?id=advanced_storyboard_control
http://dimplejs.org/advanced_examples_viewer.html?id=advanced_interactive_legends